In [1]:
#!/usr/bin/python3.7

In [2]:
import time
import sklearn
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from baggingPU import BaggingClassifierPU

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

In [3]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

# import data

In [4]:
df_raw = pd.read_csv('../data/w-independence.csv')

#df_raw = pd.read_csv('../data/1place-independence.csv')

#df_raw = pd.read_csv('../data/w-related.csv')


df_raw['label'] = df_raw['label'].astype("int")
print(df_raw.label.value_counts())
print('Has null values', df_raw.isnull().values.any())

1    10000
0    10000
Name: label, dtype: int64
Has null values False


In [5]:
df_raw.head(10)

,p1,p2,p3,label
0,0,0,1,1
1,1,0,1,1
2,0,1,1,1
3,2,0,1,1
4,1,1,1,1
5,0,1,0,1
6,0,2,1,1
7,3,0,1,1
8,2,1,1,1
9,1,1,0,1


In [6]:
print(df_raw.iloc[:10000,:])
df_max = np.amax(df_raw.iloc[:10000,[2]])
print(df_max)

      p1  p2  p3  label
0      0   0   1      1
1      1   0   1      1
2      0   1   1      1
3      2   0   1      1
4      1   1   1      1
...   ..  ..  ..    ...
9995  52  47   0      1
9996  52  48   1      1
9997  51  48   0      1
9998  51  49   1      1
9999  50  49   0      1

[10000 rows x 4 columns]
p3    1
dtype: int64


In [7]:
print(df_raw.iloc[10000:,:])
print(np.amax(df_raw.iloc[10000:,[2]]))

       p1  p2   p3  label
10000   8  41   52      0
10001  65  62  249      0
10002   4  21  509      0
10003  80  81  251      0
10004  75  12  919      0
...    ..  ..  ...    ...
19995  63  94  184      0
19996  43  65  209      0
19997  17  84  717      0
19998  69  83  269      0
19999  68  54  878      0

[10000 rows x 4 columns]
p3    999
dtype: int64


In [8]:
neg_process = df_raw.iloc[10000:,:].copy()
print(neg_process)

for i in range(10000):
    if neg_process.iloc[i,2] > df_max[0]:
        neg_process.iloc[i,2]  = int(neg_process.iloc[i,2] / df_max[0])

       p1  p2   p3  label
10000   8  41   52      0
10001  65  62  249      0
10002   4  21  509      0
10003  80  81  251      0
10004  75  12  919      0
...    ..  ..  ...    ...
19995  63  94  184      0
19996  43  65  209      0
19997  17  84  717      0
19998  69  83  269      0
19999  68  54  878      0

[10000 rows x 4 columns]


In [9]:
neg_process.drop_duplicates(inplace=True)
print(neg_process)
print(len(neg_process))
print(np.amax(neg_process))

       p1  p2   p3  label
10000   8  41   52      0
10001  65  62  249      0
10002   4  21  509      0
10003  80  81  251      0
10004  75  12  919      0
...    ..  ..  ...    ...
19995  63  94  184      0
19996  43  65  209      0
19997  17  84  717      0
19998  69  83  269      0
19999  68  54  878      0

[10000 rows x 4 columns]
10000
p1       100
p2        99
p3       999
label      0
dtype: int64


In [10]:
df_process = pd.concat([df_raw.iloc[:10000,:], neg_process])
print(df_process)

       p1  p2   p3  label
0       0   0    1      1
1       1   0    1      1
2       0   1    1      1
3       2   0    1      1
4       1   1    1      1
...    ..  ..  ...    ...
19995  63  94  184      0
19996  43  65  209      0
19997  17  84  717      0
19998  69  83  269      0
19999  68  54  878      0

[20000 rows x 4 columns]


In [11]:
x_data = df_process.iloc[:,:-1]
y_data = df_process.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [12]:
x_train_input = pd.concat([x_train.iloc[:,:], y_train],axis=1)
x_test_input = pd.concat([x_test.iloc[:,:],y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [13]:
x_train_input.head(10)

,p1,p2,p3,label
8118,36,54,1,1
10171,46,67,333,0
9399,53,43,0,1
12669,21,1,930,0
18809,29,96,858,0
13869,44,12,453,0
3320,51,7,1,1
14689,12,79,617,0
13087,21,66,519,0
15992,98,59,520,0


In [14]:
x_test_input.head(10)

,p1,p2,p3,label
14356,39,20,653,0
3439,50,8,0,1
12153,82,97,899,0
15029,47,35,549,0
18549,80,83,15,0
15762,74,61,231,0
12313,61,17,739,0
16034,88,14,924,0
11496,16,39,993,0
5653,23,51,0,1


In [15]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 5000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

print(X)
print(y)

       p1  p2   p3
8118   36  54    1
10171  46  67  333
9399   53  43    0
12669  21   1  930
18809  29  96  858
...    ..  ..  ...
13927  34  15  356
919     5  24    0
5699    0  74    0
10742  92   2  676
16921  48  46  373

[16000 rows x 3 columns]
8118     1
10171    0
9399     0
12669    0
18809    0
        ..
13927    0
919      0
5699     0
10742    0
16921    0
Name: label, Length: 16000, dtype: int64


In [16]:
pd.Series(y).value_counts()

0    12923
1     3077
Name: label, dtype: int64

In [17]:
print('- %d samples and %d features' % (X.shape))
print('- %d positive out of %d total before hiding labels' % (sum(df_process.label), len(df_process.label)))
print('- %d positive out of %d total after hiding labels' % (sum(y), len(y)))

- 16000 samples and 3 features
- 10000 positive out of 20000 total before hiding labels
- 3077 positive out of 16000 total after hiding labels


# Trainning directly

In [18]:
print('Training XGboost model ...')

import xgboost as xgb

model = xgb.XGBClassifier()

#from sklearn.linear_model import LogisticRegression
#model = LogisticRegression()


#from sklearn.neural_network import MLPClassifier
#model = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1), random_state=1,max_iter=100000)

model.fit(X, y)

print('Done')

Training XGboost model ...
Done


In [19]:
print('---- {} ----'.format('XGboost model'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- XGboost model ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive       6672.0       1405.0 
None

Precision:  1.0
Recall:  0.1739507242788164
Accuracy:  0.583
f1_score:  0.2963509808057372


# Training by bagging

In [20]:
#初始化多个分类器

from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()

from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1), random_state=1,max_iter=100000)

from sklearn import svm
model4 = svm.LinearSVC()

from sklearn.naive_bayes import MultinomialNB
model5 = MultinomialNB()

from sklearn.ensemble import RandomForestClassifier
model6 = RandomForestClassifier(n_estimators = 50,n_jobs = -1)

import xgboost as xgb
model7 = xgb.XGBClassifier()


In [21]:
model_list = [model1, model2, model3, model4, model5, model6, model7]

In [22]:
print('Training Logistics Regression model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model1,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Logistics Regression model...
Training bagging classifier...
Done!
Time: 2.267040560996975
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7901.0         22.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9972836152611434
Recall:  1.0
Accuracy:  0.998625
f1_score:  0.9986399604352127
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2072.0          5.0 
           true_positive          0.0       1923.0 
None

Precision:  0.9974066390041494
Recall:  1.0
Accuracy:  0.99875
f1_score:  0.9987016359387172
Training bagging classifier...
Done!
Time: 0.8105290990060894
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7901.0         22.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9972836152611434
Recall:  1.0
Ac

In [23]:
print('Training Decision Tree model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model2,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Decision Tree model...
Training bagging classifier...
Done!
Time: 0.8615914209949551
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive       1582.0       6495.0 
None

Precision:  1.0
Recall:  0.8041351987123933
Accuracy:  0.901125
f1_score:  0.8914356299752951
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive        295.0       1628.0 
None

Precision:  1.0
Recall:  0.8465938637545501
Accuracy:  0.92625
f1_score:  0.9169248099127006
Training bagging classifier...
Done!
Time: 0.7621539760002634
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive       1563.0       6514.0 
None

Precision:  1.0
Recall:  0.8064875572613595
Accuracy

In [24]:
print('Training MLP model directly ...')

Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model3,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training MLP model directly ...
Training bagging classifier...
Done!
Time: 13.967823404003866
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7895.0         28.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9965453423812461
Recall:  1.0
Accuracy:  0.99825
f1_score:  0.9982696823631194
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2071.0          6.0 
           true_positive          0.0       1923.0 
None

Precision:  0.9968895800933126
Recall:  1.0
Accuracy:  0.9985
f1_score:  0.9984423676012462
Training bagging classifier...
Done!
Time: 10.461912644997938
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7887.0         36.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9955626771847652
Recall:  1.0
Accuracy: 

In [25]:
print('Training SVM model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model4,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training SVM model...
Training bagging classifier...
Done!
Time: 1.5481538439926226
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7853.0         70.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9914078802013011
Recall:  1.0
Accuracy:  0.995625
f1_score:  0.9956854043392505
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2061.0         16.0 
           true_positive          0.0       1923.0 
None

Precision:  0.9917483238782878
Recall:  1.0
Accuracy:  0.996
f1_score:  0.9958570688762299
Training bagging classifier...
Done!
Time: 1.482800045007025
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7832.0         91.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9888589618021547
Recall:  1.0
Accuracy:  0.9943125


In [26]:
print('Training Naive Bayesianayes model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model5,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Naive Bayesianayes model...
Training bagging classifier...
Done!
Time: 0.888854144999641
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7650.0        273.0 
           true_positive          7.0       8070.0 
None

Precision:  0.9672779575692197
Recall:  0.999133341587223
Accuracy:  0.9825
f1_score:  0.9829476248477467
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2010.0         67.0 
           true_positive          3.0       1920.0 
None

Precision:  0.9662808253648717
Recall:  0.9984399375975039
Accuracy:  0.9825
f1_score:  0.9820971867007673
Training bagging classifier...
Done!
Time: 0.791442713001743
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7650.0        273.0 
           true_positive          7.0       8070.0 
None

Precision:  0.967277957

In [27]:
print('Training Random Forest model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model6,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Random Forest model...
Training bagging classifier...
Done!
Time: 13.91314621399215
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive        662.0       7415.0 
None

Precision:  1.0
Recall:  0.9180388758202302
Accuracy:  0.958625
f1_score:  0.9572682674928995
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive        113.0       1810.0 
None

Precision:  1.0
Recall:  0.9412376495059802
Accuracy:  0.97175
f1_score:  0.9697294401285829
Training bagging classifier...
Done!
Time: 9.240966787998332
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive        636.0       7441.0 
None

Precision:  1.0
Recall:  0.9212578927819735
Accuracy: 

In [28]:
print('Training XGboost model directly ...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model7,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training XGboost model directly ...
Training bagging classifier...
Done!
Time: 2.789147524003056
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive        142.0       7935.0 
None

Precision:  1.0
Recall:  0.9824192150550947
Accuracy:  0.991125
f1_score:  0.9911316512615538
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive         41.0       1882.0 
None

Precision:  1.0
Recall:  0.9786791471658867
Accuracy:  0.98975
f1_score:  0.9892247043363995
Training bagging classifier...
Done!
Time: 2.6828163110039895
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive        157.0       7920.0 
None

Precision:  1.0
Recall:  0.9805620898848583
Accur